In [1]:
from pydantic import BaseModel, Field

from voice_agent_flow.llms import create_pydantic_azure_openai
from voice_agent_flow.node import AgentNode
from voice_agent_flow.agents import MultiAgentRunner
from voice_agent_flow.agents import pmsg


class PoliceCallBasicInfo(BaseModel):
    """Information collected during a police call. Need to collect all the inforamation before createing this object."""
    case_location: str = Field(..., description='The location of the case')
    case_type: str = Field(..., description='The type of the case')
    description: str = Field(..., description='The description of the case')
    caller_name: str = Field(..., description='The name of the caller')
    
    
    def transfer(self) -> str:
        print("Transfer to safety_suggestion")
        return 'safety_suggestion'
    
    
class SafetySuggestionProvided(BaseModel):
    """Base on collected information, provide safety suggestion to the caller. After the caller responded to your suggestion, create this object."""
    suggestion_provided:bool = Field(..., description='Whether safety suggestion is provided to the caller')
    
    
    def transfer(self) -> str:
        print("All tasks completed.")
        return 'end'

In [2]:
model = create_pydantic_azure_openai('gpt-4o-mini')

instruction = """
You are a police call center agent(working at 110). You task is to talk with caller via telephone to collection information.
You resopnse should be berief and direct to the point.
"""


def location_reachable_on_map(location: str) -> bool:
    """Search if the location is reachable on map."""
    return True

agents = {
    "police_call_basic_info": AgentNode(
        name="police_call_basic_info",
        model=model,
        instruction=instruction,
        task_cls= PoliceCallBasicInfo,
        step_instruction=(
            "Collect basic information about the police call including case location, case type, description and caller name.) "
            "ask the question one at a time, do not ask multiple questions in one message."
            "When caller told you the location, make sure to call the tool to check if the location is reachable on map."
            "If the location is not reachable, ask the caller to provide more details about the location."
        ),
        examples=["请问您遇到什么紧急情况？ / 发生在哪里？/ 能简单描述一下吗？/ 您的姓名是？"],
        tools = [location_reachable_on_map],
        ),
    "safety_suggestion": AgentNode(
        name="safety_suggestion",
        model=model,
        instruction=instruction,
        task_cls= SafetySuggestionProvided,
        step_instruction="Based on the collected information, provide safety suggestion(A clear command to keep safe) to the caller. After the caller responded to your suggestion, create the schema.",
        examples=["请您保持冷静，.......(provide clear safety suggestion)", "请您打开窗户，保持空气流通"],
        ),
    }
      
      
runner = MultiAgentRunner(
    agents = agents, 
    entry_agent_name="police_call_basic_info", 
    ending_message="好的，我们这就派人过去处理您的情况，请您保持电话畅通！"
) 
 

In [3]:
from voice_agent_flow.agents.events import (
    AgentTextStream,
    ToolCallsOutput,
    ToolCallResult,
    StructuredOutput
)

In [4]:
memory = [
    
]

async def run(query):
    
    msg = pmsg.user(query)
    memory.append(msg)
    
    output_text = ""
    async for event in runner.run(message_history = memory):
        
        if isinstance(event.event, AgentTextStream):
            output_text += event.event.delta
            
        if isinstance(event.event, ToolCallsOutput):
            memory.append(pmsg.tool_call(
                tool_name = event.event.message['tool_name'],
                args = event.event.message['args'],
                tool_call_id=event.event.message['tool_call_id']
            ))
            
        if isinstance(event.event, ToolCallResult):
            memory.append(pmsg.tool_return(
                tool_name = event.event.message['tool_name'],
                content = event.event.message['content'],
                tool_call_id=event.event.message['tool_call_id']
            ))
            
        if isinstance(event.event, StructuredOutput):
            print(event.event)
            
    if len(output_text) > 0:
        memory.append(pmsg.assistant(output_text))
        print(output_text)

In [5]:
await run("你好，我家进水了")

请问发生在哪里？


In [6]:
await run("朝阳区，麦子店32号")

请问这个情况属于什么类型的案件？


In [7]:
await run("房屋受损，危险")

能简单描述一下具体情况吗？


In [8]:
await run("阀门坏了，厕所一直漏水")

您的姓名是？


In [9]:
await run("王小明")

Transfer to safety_suggestion
StructuredOutput(status=None, message=PoliceCallBasicInfo(case_location='朝阳区，麦子店32号', case_type='房屋受损', description='阀门坏了，厕所一直漏水', caller_name='王小明'))


In [10]:
await run("嗯")

请您立即关闭水阀，避免更大的水灾。如果有必要，尽快联系专业的水管工进行修理。保持手机畅通，等候进一步指导。


In [11]:
await run("好的，再见")

All tasks completed.
好的，我们这就派人过去处理您的情况，请您保持电话畅通！


In [12]:
memory

[ModelRequest(parts=[UserPromptPart(content='你好，我家进水了', timestamp=datetime.datetime(2026, 2, 21, 15, 53, 34, 44340, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[TextPart(content='请问发生在哪里？')], usage=RequestUsage(), timestamp=datetime.datetime(2026, 2, 21, 15, 53, 35, 836511, tzinfo=datetime.timezone.utc)),
 ModelRequest(parts=[UserPromptPart(content='朝阳区，麦子店32号', timestamp=datetime.datetime(2026, 2, 21, 15, 53, 35, 843885, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[ToolCallPart(tool_name='location_reachable_on_map', args='{"location":"朝阳区，麦子店32号"}', tool_call_id='call_aK3mdeMd6T69G5NrqJ9kI0vq')], usage=RequestUsage(), timestamp=datetime.datetime(2026, 2, 21, 15, 53, 37, 706245, tzinfo=datetime.timezone.utc)),
 ModelRequest(parts=[ToolReturnPart(tool_name='location_reachable_on_map', content=True, tool_call_id='call_aK3mdeMd6T69G5NrqJ9kI0vq', timestamp=datetime.datetime(2026, 2, 21, 15, 53, 37, 707304, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[TextPa